<a name="top"></a>
# A land use classification for Fray Jorge
Developed by Diego Ocampo Melgar

[Contact](diego.ocampo.melgar@gmail.com)

## Index

[Dependencias](#setup)

[Configuración](#config)

[Loop de entrenamiento por clases](#loop)

[Integrar datos](#join)

[Limpieza de datos](#clean)

[Diseño del modelo FNN](#model)

[Entrenamiento del modelo](#train)

[Desempeño del modelo](#performance)

[Predicción](#predict)

[Visualización](#view)


[Fuente](https://chat.deepseek.com/a/chat/s/b31ae746-5a2a-4a17-800f-2c162618c5e6)

# Unsupervised Classification of Raster Stacks V1


In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from sklearn.cluster import (KMeans, MiniBatchKMeans, DBSCAN,
                           AgglomerativeClustering, SpectralClustering)
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load your raster stack (example structure)
# ds = xr.open_dataset('your_raster_stack.nc')
# Let's assume ds has dimensions (time, y, x) and multiple variables

# 1. Data Preparation
def prepare_data(ds):
    """Convert xarray Dataset to 2D array for clustering"""
    # Stack all variables and time steps
    data = ds.to_array().values  # (variables, time, y, x)
    # Reshape to (n_samples, n_features)
    n_samples = data.shape[2] * data.shape[3]  # y * x
    n_features = data.shape[0] * data.shape[1]  # variables * time
    return data.reshape(n_features, -1).T  # (n_samples, n_features)

X = prepare_data(ds)

# 2. Dimensionality Reduction (optional but recommended)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=0.95)  # Keep 95% variance
X_pca = pca.fit_transform(X_scaled)

# 3. Define Clustering Models to Evaluate
models = {
    'KMeans (k=5)': KMeans(n_clusters=5, random_state=42),
    'MiniBatchKMeans (k=5)': MiniBatchKMeans(n_clusters=5, random_state=42),
    'DBSCAN (eps=0.5)': DBSCAN(eps=0.5, min_samples=5),
    'Agglomerative (k=5)': AgglomerativeClustering(n_clusters=5),
    'Spectral (k=5)': SpectralClustering(n_clusters=5, random_state=42)
}

# 4. Evaluate Models
results = []
cluster_maps = {}

for name, model in models.items():
    print(f"Running {name}...")

    # Fit model
    labels = model.fit_predict(X_pca if 'Spectral' not in name else X_scaled)

    # Skip metrics for DBSCAN if too many noise points
    if isinstance(model, DBSCAN) and (labels == -1).mean() > 0.5:
        print(f"Skipping {name} - too many noise points")
        continue

    # Calculate metrics (only if not all noise)
    if len(np.unique(labels)) > 1:
        metrics = {
            'Model': name,
            'Silhouette': silhouette_score(X_pca, labels),
            'Calinski-Harabasz': calinski_harabasz_score(X_pca, labels),
            'Davies-Bouldin': davies_bouldin_score(X_pca, labels),
            'n_clusters': len(np.unique(labels))
        }
        results.append(metrics)

    # Store labels for visualization
    cluster_maps[name] = labels.reshape(ds.dims['y'], ds.dims['x'])

# 5. Performance Comparison
results_df = pd.DataFrame(results).set_index('Model')
print("\nPerformance Metrics:")
print(results_df)

# Plot metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for i, metric in enumerate(['Silhouette', 'Calinski-Harabasz', 'Davies-Bouldin']):
    sns.barplot(data=results_df.reset_index(), x='Model', y=metric, ax=axes[i])
    axes[i].set_title(metric)
    axes[i].tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

# 6. Visualize Cluster Maps
n_models = len(cluster_maps)
fig, axes = plt.subplots(1, n_models, figsize=(5*n_models, 5))
if n_models == 1:
    axes = [axes]

for ax, (name, labels) in zip(axes, cluster_maps.items()):
    im = ax.imshow(labels, cmap='viridis')
    ax.set_title(name)
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.show()

# 7. Optimal Cluster Number Analysis (for KMeans)
if 'KMeans (k=5)' in models:
    silhouette_scores = []
    calinski_scores = []
    k_values = range(2, 11)

    for k in k_values:
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X_pca)
        silhouette_scores.append(silhouette_score(X_pca, labels))
        calinski_scores.append(calinski_harabasz_score(X_pca, labels))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    ax1.plot(k_values, silhouette_scores, 'bo-')
    ax1.set_xlabel('Number of clusters')
    ax1.set_ylabel('Silhouette Score')
    ax1.set_title('Silhouette Method')

    ax2.plot(k_values, calinski_scores, 'go-')
    ax2.set_xlabel('Number of clusters')
    ax2.set_ylabel('Calinski-Harabasz Score')
    ax2.set_title('Variance Ratio Method')
    plt.tight_layout()
    plt.show()

Key Components:
* Data Preparation:

  - Converts xarray Dataset to 2D array (pixels × features)
  - Handles multiple variables and time steps

* Dimensionality Reduction:

  - Standard scaling for normalization

  - PCA to reduce computational complexity

* Model Architectures:

  - KMeans: Classic centroid-based clustering

  - MiniBatchKMeans: Faster version for large datasets

  - DBSCAN: Density-based clustering

  - Agglomerative: Hierarchical clustering

  - Spectral: Graph-based clustering

* Performance Metrics:

  - Silhouette Score (-1 to 1, higher is better)

  - Calinski-Harabasz (higher is better)

  - Davies-Bouldin (lower is better)

* Visualization:

  - Cluster maps for spatial patterns

  - Metric comparison bar plots

  - Elbow plots for optimal cluster number

##Alternative Model Options:




1.   Gaussian Mixture Models:



In [ ]:
from sklearn.mixture import GaussianMixture
models['GMM (k=5)'] = GaussianMixture(n_components=5, random_state=42)

2. HDBSCAN (improved density-based):

In [ ]:
from hdbscan import HDBSCAN
models['HDBSCAN'] = HDBSCAN(min_cluster_size=5)

3. Self-Organizing Maps:

In [ ]:
from minisom import MiniSom
som = MiniSom(5, 5, X_pca.shape[1], sigma=0.5, learning_rate=0.5)
som.train_random(X_pca, 100)

##Performance Optimization Tips:
* For large rasters:

  - Use MiniBatchKMeans or subsample

  - Reduce PCA components

  - Process in chunks

* For better spatial coherence:

  - Include spatial coordinates as features

  - Use SpatialKMeans from sklearn-extensions

* For temporal patterns:

  - Extract temporal features before clustering

  - Use TimeSeriesKMeans from tslearn

This framework provides a complete pipeline from data preparation to model evaluation, with multiple visualization options to interpret the results. The choice of best model depends on your specific data characteristics and application requirements.

## Unsupervised Model Comparison Summary

# Unsupervised Classification Models Comparison

| Model               | Pros                                                                 | Cons                                                                 | Best For                                  |
|---------------------|----------------------------------------------------------------------|----------------------------------------------------------------------|-------------------------------------------|
| **KMeans**          | ✅ Fast computation<br>✅ Scalable to large data<br>✅ Guaranteed convergence | ❌ Requires predefined *k*<br>❌ Sensitive to initialization<br>❌ Assumes spherical clusters | Homogeneous landscapes with clear spectral separation |
| **MiniBatchKMeans** | ✅ Faster than KMeans for large datasets<br>✅ Memory-efficient       | ❌ Slightly lower accuracy<br>❌ Same limitations as KMeans          | Large raster stacks (>1M pixels)          |
| **DBSCAN**          | ✅ No need to specify *k*<br>✅ Handles irregular shapes<br>✅ Identifies noise | ❌ Sensitive to parameters (*eps*, *min_samples*)<br>❌ Struggles with varying densities | Heterogeneous landscapes with natural clusters |
| **Agglomerative**   | ✅ Hierarchical structure<br>✅ Flexible distance metrics<br>✅ Visualizable dendrograms | ❌ Computationally expensive (O(n³))<br>❌ Memory-intensive | Small datasets with hierarchical patterns |
| **Spectral**        | ✅ Captures complex structures<br>✅ Works with non-convex shapes     | ❌ Requires affinity matrix (memory-heavy)<br>❌ Slow for large datasets | High-dimensional data with non-linear patterns |
| **GMM**             | ✅ Soft clustering (probabilities)<br>✅ Flexible cluster shapes      | ❌ Sensitive to initialization<br>❌ Assumes Gaussian distributions  | Probabilistic classification needs        |


### **Key Evaluation Metrics**
| Metric               | Range          | Interpretation                          |
|----------------------|----------------|-----------------------------------------|
| **Silhouette Score** | -1 to 1        | Higher = Better separation              |
| **Calinski-Harabasz**| 0 to ∞         | Higher = Better defined clusters        |
| **Davies-Bouldin**   | 0 to ∞         | Lower = Better cluster separation       |


*Practical Recommendations*

For large datasets:

* Start with MiniBatchKMeans (speed) or HDBSCAN (auto-clustering)

For noisy data:

* Use DBSCAN or HDBSCAN to filter noise

For hierarchical patterns:

* Agglomerative Clustering with linkage plots

When k is unknown:

* Run KMeans with elbow/silhouette analysis first

For spatial coherence:

* Add XY coordinates as features or use SpatialKMeans

## Visualization Workflow

In [ ]:
# PCA Scatter Plot:
plt.scatter(X_pca[:,0], X_pca[:,1], c=labels, cmap='viridis', s=1)

In [ ]:
# Cluster map:
plt.imshow(labels.reshape(ds.y.size, ds.x.size), cmap='tab20')

In [ ]:
# Dendrogram:
from scipy.cluster.hierarchy import dendrogram
dendrogram(model.children_, truncate_mode='level', p=3)

### **Performance Benchmark Example**
| Model               | Silhouette | Calinski-Harabasz | Davies-Bouldin | Time (s) |
|---------------------|------------|-------------------|----------------|----------|
| KMeans (k=5)        | 0.62       | 1204              | 0.81           | 12.1     |
| DBSCAN (eps=0.3)    | 0.58       | 984               | 0.92           | 8.7      |
| Spectral (k=5)      | 0.65       | 1350              | 0.75           | 42.3     |


Trade-off: Spectral clustering often performs best but is 3-4x slower than KMeans. For time-critical applications, MiniBatchKMeans provides a good balance.

### **Recommendations**
1. **For computational efficiency**: `MiniBatchKMeans`  
2. **For automatic cluster detection**: `DBSCAN`/`HDBSCAN`  
3. **For hierarchical relationships**: `Agglomerative` + dendrogram  
4. **For spatial coherence**: Include XY coordinates as features

### Save and predict


In [ ]:
# Training Phase (Reference Year)
import numpy as np
import xarray as xr
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import joblib  # for model saving

# Load reference year data
ds_train = xr.open_dataset('2018_raster.nc')  # Your training year
X_train = ds_train.to_array().values.reshape(ds_train.dims['variable'], -1).T

# Preprocess
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Train model
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X_train_scaled)

# Save artifacts
joblib.dump(kmeans, 'kmeans_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

In [ ]:
# Prediction Phase (New Year)
# Load new year data
ds_test = xr.open_dataset('2022_raster.nc')  # New year to classify
X_test = ds_test.to_array().values.reshape(ds_test.dims['variable'], -1).T

# Load saved models
kmeans = joblib.load('kmeans_model.pkl')
scaler = joblib.load('scaler.pkl')

# Predict clusters
X_test_scaled = scaler.transform(X_test)  # Use same scaling!
test_labels = kmeans.predict(X_test_scaled)

# Reshape to original raster
cluster_map = test_labels.reshape(ds_test.dims['y'], ds_test.dims['x'])

*Key Considerations*
- Feature Alignment:	Ensure same bands/variables in same order, Unsupervised models don't generalize to new features.
- Scaling:	Use the same scaler from training to prevent cluster distortion from different value ranges.
- Dimensionality:	Apply identical PCA if used during training. Maintains the same feature space.
- Cluster Interpretation:	Compare class statistics between years to verify ecological consistency of clusters.


### Alternative Approaches

In [ ]:

# A. Direct Transfer (Best for stable environments)
## Simply apply the trained model to new data
labels_new = kmeans.predict(new_data_scaled)

# B. Fine-Tuning (For gradual changes)
## Use previous clusters as initialization
kmeans_new = KMeans(n_clusters=5, init=kmeans.cluster_centers_, n_init=1)
kmeans_new.fit(new_data_scaled)

# C. Pseudo-Labeling (When some ground truth exists)
from sklearn.semi_supervised import SelfTrainingClassifier
base_model = KMeans(n_clusters=5)
st_classifier = SelfTrainingClassifier(base_model)
st_classifier.fit(partial_labeled_data)

* Validation Methods:

In [ ]:
#Spectral Signature Comparison
# Compare mean band values per cluster between years
pd.DataFrame({
    '2018': ds_train.groupby(cluster_map_train).mean(),
    '2022': ds_test.groupby(cluster_map_test).mean()
})

In [ ]:
#Spatial Consistency Check
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))
ax1.imshow(cluster_map_train, cmap='tab20')
ax2.imshow(cluster_map_test, cmap='tab20')

In [ ]:
#Change Detection
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(cluster_map_train.flatten(), cluster_map_test.flatten())
sns.heatmap(cm, annot=True)

**When This Works Best**
- Temporal consistency: Similar phenology/conditions between years

- Stable features: Same sensors/bands with consistent radiometry

- Moderate changes: Gradual land cover evolution rather than abrupt changes

For radically different conditions (e.g., wildfire year), consider retraining or using ensemble methods. The approach works well for applications like:

- Annual crop type mapping

- Urban expansion monitoring

- Seasonal vegetation patterns

[Source](https://https://chat.deepseek.com/a/chat/s/b31ae746-5a2a-4a17-800f-2c162618c5e6)

# Unsupervised Classification of Raster Stacks V2

| Model                            | Type          | Description                                      |
| -------------------------------- | ------------- | ------------------------------------------------ |
| **KMeans**                       | Centroid      | Assigns each point to the nearest cluster center |
| **Gaussian Mixture Model (GMM)** | Probabilistic | Models data as mixtures of Gaussians             |
| **Spectral Clustering**          | Graph-based   | Uses eigenvectors of similarity matrix           |
| **DBSCAN**                       | Density       | Groups based on density, robust to noise         |
| **Self-Organizing Map (SOM)**    | Neural        | Neural network mapping to a low-dimensional grid |
| **Autoencoder + KMeans**         | Deep Hybrid   | Compress data, then cluster in latent space      |


## Preprocessing



In [ ]:
import xarray as xr
import numpy as np
from sklearn.preprocessing import StandardScaler

# Convert to DataFrame
ds = xr.open_dataset("your_stack.nc")
arr = ds.to_array().values  # (bands, y, x)
arr_2d = arr.reshape(arr.shape[0], -1).T  # (pixels, bands)

# Normalize
scaler = StandardScaler()
X = scaler.fit_transform(arr_2d)


## Clustering Options

### A. K-means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=0)
labels_kmeans = kmeans.fit_predict(X)


### B. Gaussian Mixture Model

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=5, covariance_type='full', random_state=0)
labels_gmm = gmm.fit_predict(X)


### C. DBSCAN (with tuned eps)

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.5, min_samples=10)
labels_dbscan = dbscan.fit_predict(X)


D. Autoencoder + KMeans (optional)

Use tensorflow.keras to reduce dimensionality, then apply KMeans.

## Evaluation Metrics (Unsupervised)

Since there's no ground truth, we rely on intrinsic clustering quality metrics:

In [ ]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

def evaluate(X, labels):
    print("Silhouette Score:", silhouette_score(X, labels))
    print("Calinski-Harabasz Score:", calinski_harabasz_score(X, labels))
    print("Davies-Bouldin Score:", davies_bouldin_score(X, labels))


## Plotting
### Reshape back to image

In [ ]:
import matplotlib.pyplot as plt

height, width = arr.shape[1:]
labels_img = labels_kmeans.reshape(height, width)

plt.imshow(labels_img, cmap="tab10")
plt.title("KMeans Classification")
plt.axis('off')
plt.show()


### PCA Plot

In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

df_plot = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_plot["Cluster"] = labels_kmeans

sns.scatterplot(data=df_plot, x="PC1", y="PC2", hue="Cluster", palette="tab10")
plt.title("Cluster Visualization (PCA)")
plt.show()


## Summary of Models

| Model                   | Pros                                     | Cons                                                   |
| ----------------------- | ---------------------------------------- | ------------------------------------------------------ |
| **KMeans**              | Fast, easy to implement                  | Assumes spherical clusters, sensitive to `k`           |
| **GMM**                 | Probabilistic, handles elliptical shapes | Computationally expensive, sensitive to initialization |
| **DBSCAN**              | No need for `k`, detects noise           | Poor in high-dimensions, sensitive to `eps`            |
| **Spectral Clustering** | Good with non-convex clusters            | Slow on large datasets                                 |
| **SOM**                 | Interpretable, maps to grid              | Needs careful tuning, less common                      |
| **Autoencoder+KMeans**  | Learns latent structure, robust          | Complex to train, risk of overfitting                  |



### Recommendation


If your raster stack is moderately sized, try:

- KMeans and GMM for quick and reliable results.

- DBSCAN if you expect noise and irregular clusters.

- Autoencoder + KMeans if the stack is large or you suspect nonlinear structure.

- Use Silhouette Score and Calinski-Harabasz for comparison.

## Full pipeline

In [ ]:
# Unsupervised Classification on Raster Stack using xarray, scikit-learn, and Autoencoder

import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Load Raster Stack (assuming NetCDF format)
ds = xr.open_dataset("your_raster_stack.nc")  # Replace with your file path
arr = ds.to_array().values  # Shape: (bands, y, x)
bands, height, width = arr.shape
arr_2d = arr.reshape(bands, -1).T  # Shape: (pixels, bands)

# Remove NaNs
valid_mask = ~np.any(np.isnan(arr_2d), axis=1)
X_valid = arr_2d[valid_mask]

# Normalize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_valid)

# Function to evaluate clustering

def evaluate_clustering(X, labels, name):
    print(f"\n--- {name} ---")
    print("Silhouette Score:", silhouette_score(X, labels))
    print("Calinski-Harabasz Score:", calinski_harabasz_score(X, labels))
    print("Davies-Bouldin Score:", davies_bouldin_score(X, labels))

# 1. KMeans
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans_labels = kmeans.fit_predict(X_scaled)
evaluate_clustering(X_scaled, kmeans_labels, "KMeans")

# 2. Gaussian Mixture Model
gmm = GaussianMixture(n_components=5, covariance_type='full', random_state=42)
gmm_labels = gmm.fit_predict(X_scaled)
evaluate_clustering(X_scaled, gmm_labels, "GMM")

# 3. DBSCAN (parameters may need tuning)
dbscan = DBSCAN(eps=0.5, min_samples=10)
dbscan_labels = dbscan.fit_predict(X_scaled)

# Filter out noise (-1 labels)
if len(set(dbscan_labels)) > 1 and -1 in dbscan_labels:
    print("\nDBSCAN filtered noise")
    evaluate_clustering(X_scaled[dbscan_labels != -1], dbscan_labels[dbscan_labels != -1], "DBSCAN")
else:
    print("\nDBSCAN did not identify more than one cluster.")

# 4. Autoencoder-based Feature Extraction
def build_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dense(32, activation='relu')(encoded)
    encoded = Dense(10, activation='relu')(encoded)
    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(decoded)
    output_layer = Dense(input_dim, activation='linear')(decoded)
    autoencoder = Model(input_layer, output_layer)
    encoder = Model(input_layer, encoded)
    return autoencoder, encoder

input_dim = X_scaled.shape[1]
autoencoder, encoder = build_autoencoder(input_dim)
autoencoder.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
autoencoder.fit(X_scaled, X_scaled, epochs=100, batch_size=256, shuffle=True, callbacks=[early_stop], verbose=0)

X_encoded = encoder.predict(X_scaled)

# Clustering on Autoencoder-encoded Features
auto_kmeans = KMeans(n_clusters=5, random_state=42)
auto_kmeans_labels = auto_kmeans.fit_predict(X_encoded)
evaluate_clustering(X_encoded, auto_kmeans_labels, "Autoencoder + KMeans")

# PCA for visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
X_enc_pca = pca.fit_transform(X_encoded)

df_plot = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_plot["KMeans"] = kmeans_labels
df_plot["GMM"] = gmm_labels
if len(set(dbscan_labels)) > 1:
    df_plot["DBSCAN"] = dbscan_labels
df_plot_auto = pd.DataFrame(X_enc_pca, columns=["PC1", "PC2"])
df_plot_auto["Auto+KMeans"] = auto_kmeans_labels

# Plotting clusters
plt.figure(figsize=(20, 4))
methods = ["KMeans", "GMM"] + (["DBSCAN"] if "DBSCAN" in df_plot else [])
for i, method in enumerate(methods):
    plt.subplot(1, len(methods) + 1, i + 1)
    sns.scatterplot(data=df_plot, x="PC1", y="PC2", hue=method, palette="tab10", s=10, linewidth=0)
    plt.title(f"{method} Clusters")
    plt.legend(loc='best', fontsize='small')

plt.subplot(1, len(methods) + 1, len(methods) + 1)
sns.scatterplot(data=df_plot_auto, x="PC1", y="PC2", hue="Auto+KMeans", palette="tab10", s=10, linewidth=0)
plt.title("Autoencoder + KMeans")
plt.legend(loc='best', fontsize='small')

plt.tight_layout()
plt.show()

# Reconstruct cluster image for visualization (Autoencoder + KMeans)
labels_full = np.full(arr_2d.shape[0], -1)
labels_full[valid_mask] = auto_kmeans_labels
img_auto_kmeans = labels_full.reshape(height, width)

plt.imshow(img_auto_kmeans, cmap="tab10")
plt.title("Autoencoder + KMeans Cluster Map")
plt.axis('off')
plt.show()

